In [23]:
# Imports for handling time series 
import numpy as np
import matplotlib.pyplot as plt 
from scipy import signal 
import scipy.io as sio
import sys
import os
import importlib
import datetime as dt

# Add path to tsa_lth library
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'TimeSeriesAnalysis-main', 'TimeSeriesAnalysis-main')))

# Imports from the course library 
import tsa_lth.analysis
import tsa_lth.modelling
import tsa_lth.tests
importlib.reload(tsa_lth.analysis)
importlib.reload(tsa_lth.modelling)
importlib.reload(tsa_lth.tests)

from tsa_lth.analysis import plotACFnPACF, normplot, xcorr, pzmap, kovarians, tacf, box_cox
from tsa_lth.modelling import estimateARMA, polydiv, PEM 
from tsa_lth.tests import whiteness_test, check_if_normal

In [3]:
# Choose one or the other, for convenience 
%matplotlib qt

In [ ]:
%matplotlib inline

In [104]:
# ============================
# ========== PART A ==========
# ============================

# Load data 
filename = "../data/projectData25.mat"
data = sio.loadmat(filename)['data']

# Extract timestamps from the last columns 
time = np.array([f'{int(data[idx, 4]):04d}-{int(data[idx, 5]):02d}-{int(data[idx, 6]):02d}T{int(data[idx, 7]):02d}' 
                 for idx in range(data.shape[0])], dtype=np.datetime64)

# There is an issue with the timestamps during model data at index 895, fix it! 
time[895] = np.datetime64('1989-08-13T00')

# Plot data to get an overview
# Note, the columns are Obs nr, Power [MJ/s], Ambient temp [C], Supply temp [C] and time Year/Month/Day/Hour
fig, axs = plt.subplots(3, 1, layout='constrained', sharex=True)
axs[0].plot(time, data[:, 1])
axs[0].set_ylabel('Power [MJ/s]')
axs[0].set_title('Power Load')
axs[1].plot(time, data[:, 2])
axs[1].set_ylabel('Temp [$^\circ$C]')
axs[1].set_title('Ambient Temperature')
axs[2].plot(time, data[:, 3])
axs[2].set_ylabel('Temp [$^\circ$C]')
axs[2].set_title('Supply Water Temperature')
axs[2].set_xlabel('Time')
axs[2].tick_params(axis='x', labelrotation=45)   
    
# Power should probably be transformed 

In [105]:
# Extract a date range for modelling, validation and testing 
# Based on the plotted data, we have reasonably stationary data during late summer 
# We could start eg 07-15. Winter also appear stationary, however there is a long sequence of missing data 

t0 = np.datetime64('1989-07-01T00')
t1 = np.datetime64('1989-09-10T00')
t2 = np.datetime64('1989-09-24T00')
t3 = np.datetime64('1989-10-01T00')

model_start = np.searchsorted(time, t0)
val_start = np.searchsorted(time, t1)
test_start = np.searchsorted(time, t2)
test_end = np.searchsorted(time, t3)

fig, axs = plt.subplots(3, 1, layout='constrained', sharex=True)
axs[0].plot(time[model_start:val_start], data[model_start:val_start, 1], color='C0', label='Model')
axs[0].plot(time[val_start:test_start], data[val_start:test_start, 1], color='C1', label='Validation')
axs[0].plot(time[test_start:test_end], data[test_start:test_end, 1], color='C2', label='Test')
axs[0].set_ylabel('Power [MJ/s]')
axs[0].set_title('Power Load')
axs[0].legend()
axs[1].plot(time[model_start:val_start], data[model_start:val_start, 2], color='C0')
axs[1].plot(time[val_start:test_start], data[val_start:test_start, 2], color='C1')
axs[1].plot(time[test_start:test_end], data[test_start:test_end, 2], color='C2')
axs[1].set_ylabel('Temp [$^\circ$C]')
axs[1].set_title('Ambient Temperature')
axs[2].plot(time[model_start:val_start], data[model_start:val_start, 3], color='C0')
axs[2].plot(time[val_start:test_start], data[val_start:test_start, 3], color='C1')
axs[2].plot(time[test_start:test_end], data[test_start:test_end, 3], color='C2')
axs[2].set_ylabel('Temp [$^\circ$C]')
axs[2].set_title('Supply Water Temperature')
axs[2].set_xlabel('Time')
axs[2].tick_params(axis='x', labelrotation=45) 

# Also pick a time period during winter 
tw0 = np.datetime64('1990-02-01T00')
tw1 = np.datetime64('1990-02-08T00')
wint_start = np.searchsorted(time, tw0)
wint_end = np.searchsorted(time, tw1)


In [8]:
# Start Modelling! We will create a Box-Jenkins model for A using the transfer function approach 
# Save variables for faster access 
power = data[:, 1]
ambient = data[:, 2]

In [9]:
# Pre-whitening of the input process 
ambient_model = ambient[model_start:val_start]
power_model = power[model_start:val_start]
# PACF and ACF
plotACFnPACF(ambient_model, noLags=40, signLvl=0.05)

print('\n===== ARMA(2,0) =====\n')

# Two clear spiken in PACF, ACF declines very slowly and ringing. AR2? 
p3 = 2
q3 = 0
input_model_1 = estimateARMA(ambient_model, A=p3, C=q3, plot=False)
wt_1 = signal.lfilter(input_model_1.A, input_model_1.C, ambient_model)[p3:]
epst_1 = signal.lfilter(input_model_1.A, input_model_1.C, power_model)[p3:]

# Study modelling results 
plotACFnPACF(wt_1, noLags=40, signLvl=0.05)
whiteness_test(wt_1)
plt.figure()
normplot(wt_1)
check_if_normal(wt_1)

# Conclusion, not white. Clear spike for lag 3 in ACF. Maybe a small 24 hour period? Start with the former 

print('\n===== ARMA(2,3) =====\n')

p3 = 2
q3 = 3
cf = np.array([1, 0, 0, 1])
input_model_2 = estimateARMA(ambient_model, A=p3, C=q3, plot=False)
wt_2 = signal.lfilter(input_model_2.A, input_model_2.C, ambient_model)[p3:]
epst_2 = signal.lfilter(input_model_2.A, input_model_2.C, power_model)[p3:]

# Study modelling results 
plotACFnPACF(wt_2, noLags=40, signLvl=0.05)
whiteness_test(wt_2)
plt.figure()
normplot(wt_2)
check_if_normal(wt_2)

# Not much better even with a full ARMA(3,3), so we will need to explore the periodicity better
# Based on shape in PACF and logic, 24 hours = 24 samples seems reasonable.

print('\n===== SARIMA(2,0) =====\n')

diff = np.array([1] + [0]*23 + [-1])
ambient_diff = signal.lfilter(diff, [1], ambient_model)[diff.size:]
p3 = 2
q3 = 0
input_model_3 = estimateARMA(ambient_diff, A=p3, C=q3, plot=False)
wt_3 = signal.lfilter(input_model_3.A, input_model_3.C, ambient_diff)[p3:]
epst_3 = signal.lfilter(input_model_3.A, input_model_3.C, ambient_diff)[p3:]

# Study modelling results 
plotACFnPACF(wt_3, noLags=40, signLvl=0.05)
whiteness_test(wt_3)
plt.figure()
normplot(wt_3)
check_if_normal(wt_3)

print('\n===== SARIMA(2,24) =====\n')

p3 = 3
q3 = 24
cf = np.array([1, 0, 0, 1] + [0]*20 + [1])
input_model_4 = estimateARMA(ambient_diff, A=p3, C=q3, C_free=cf, plot=False)
wt_4 = signal.lfilter(input_model_4.A, input_model_4.C, ambient_diff)[p3:]
epst_4 = signal.lfilter(input_model_4.A, input_model_4.C, ambient_diff)[p3:]

# Study modelling results 
plotACFnPACF(wt_4, noLags=40, signLvl=0.05)
whiteness_test(wt_4)
plt.figure()
normplot(wt_4)
check_if_normal(wt_4)


===== ARMA(2,0) =====

Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 324.01 < 37.65)
  McLeod-Li test:        False (white if 197.71 < 37.65)
  Monti test:            False (white if 338.39 < 37.65)
  Sign change test:      True (white if 0.49 in [0.48,0.52])
The D'Agostino-Pearson K2 test indicates that the data is NOT normal distributed.

===== ARMA(2,3) =====

Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 301.61 < 37.65)
  McLeod-Li test:        False (white if 196.32 < 37.65)
  Monti test:            False (white if 322.60 < 37.65)
  Sign change test:      True (white if 0.48 in [0.48,0.52])
The D'Agostino-Pearson K2 test indicates that the data is NOT normal distributed.

===== SARIMA(2,0) =====

Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 417.72 < 37.65)
  McLeod-Li test:        False (white if 480.31 < 37.65)
  Monti test:            False (white if 419.13 < 37.65)
  Sign change

In [41]:
box_cox(power[model_start:val_start])
box_cox(ambient[model_start:val_start])

Max Lambda = 0.491917917905124.
y^0.5 could be an appropriate transformation.
Max Lambda = -0.7833275698346761.
y^-1 could be an appropriate transformation.


In [160]:
# Save the current best below
print('===== Current Best =====')

# Try transformations according to box cox 
amb_tfm = ambient ** (-1)
pow_tfm = np.sqrt(power)

diff = np.array([1] + [0]*23 + [-1])
ambient_diff = signal.lfilter(diff, [1], amb_tfm)
power_diff = signal.lfilter(diff, [1], pow_tfm)
start = max(24, model_start)
ambient_diff_mdl = ambient_diff[start:val_start]
power_diff_mdl = power_diff[start:val_start]

fix, ax = plt.subplots() 
ax.plot(ambient_diff_mdl, label='amb')
ax.plot(power_diff_mdl, label='pow')
ax.legend()

p3 = 2
q3 = 24
cf = np.array([1, 0, 0, 1] + [0]*20 + [1])
#ad = np.array([1] + [0]*23 + [1])
#a2 = np.array([1, 1, 1])
#af = np.convolve(ad, a2)

input_model = estimateARMA(ambient_diff_mdl, A=p3, C=q3, C_free=cf, plot=False)
wt = signal.lfilter(input_model.A, input_model.C, ambient_diff_mdl)[24:]
eps_t = signal.lfilter(input_model.A, input_model.C, power_diff_mdl)[24:]

# Study modelling results 
plotACFnPACF(wt, noLags=40, signLvl=0.05)
whiteness_test(wt)
plt.figure()
normplot(wt)
check_if_normal(wt)

wt_tacf = tacf(wt)
conf = 2 / np.sqrt(ambient_diff.size)
fig, ax = plt.subplots() 
ax.stem(wt_tacf)
ax.axhline(y=conf, color='red', linestyle='--')
ax.axhline(y=-conf, color='red', linestyle='--')

# Note however that the Monti test is still not satisfied 
# Note: The larger values are present even in the TACF; hence outliers are an unliekly cause 
# Note II: Does the variability increase as the temp increases? Transform? 
# Note III: A Box-Cox plot suggests a transform may be useful, however, the resulting model is not better. 
# Note IV: Add differentiation? We are in early autumn, so we shold see a declining trend. 


===== Current Best =====
Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 72.34 < 37.65)
  McLeod-Li test:        False (white if 292.74 < 37.65)
  Monti test:            False (white if 84.36 < 37.65)
  Sign change test:      True (white if 0.48 in [0.47,0.53])
The D'Agostino-Pearson K2 test indicates that the data is NOT normal distributed.


In [161]:
# Proceed with the above model for now, try to refine it later if possible/necessary 

# Use this from lab2 
def compute_ccf(x, y, maxlag):
    Cxy = np.correlate(y - np.mean(y), x - np.mean(x), mode='full')
    Cxy = Cxy / (np.std(y) * np.std(x) * len(y))
    lags = np.arange(-maxlag, maxlag + 1)
    mid = len(Cxy) // 2
    Cxy = Cxy[mid - maxlag:mid + maxlag + 1]
    return lags, Cxy

lags, ccf = compute_ccf(wt, eps_t, 40)
fig, ax = plt.subplots(layout='constrained')
ax.stem(lags, ccf, basefmt=' ')
condInt = 2 / np.sqrt(len(wt))
ax.axhline(condInt, color='r', linestyle='--', label='95% confidence')
ax.axhline(-condInt, color='r', linestyle='--')
ax.set_xlabel('Lag')
ax.set_ylabel('CCF')
ax.set_title('Cross-correlation between w_t and eps_t')
plt.grid(True)
plt.show()

# Cross-correlation shows one major spike for lag zero. This could indicate that only the current 
# temperature is relevant for modelling the power output, i.e. r = s = d = 0
# Log looks better 

In [162]:
# We proceed with this hypothesis 
B_init = np.array([1])
A2_init = np.array([1, 1])
A1_init = np.array([1])
C1_init = np.array([1])

# Same part are also free 
B_free = np.array([1])
A2_free = np.array([1, 1])
A1_free = np.array([1])
C1_free = np.array([1])

# Calculate model 
pemslv1 = PEM(power_diff_mdl, ambient_diff_mdl, B=B_init, F=A2_init, C=C1_init, D=A1_init)
pemslv1.set_free_params(B_free=B_free, F_free=A2_free, C_free=C1_free, D_free=A1_free)
model_1 = pemslv1.fit()
etilde = model_1.resid
model_1.summary()

# Is etilde and x uncorrelated? 
lags, ccf = compute_ccf(etilde, ambient_diff_mdl, 40)
fig, ax = plt.subplots(layout='constrained')
ax.stem(lags, ccf, basefmt=' ')
condInt = 2 / np.sqrt(len(wt))
ax.axhline(condInt, color='r', linestyle='--', label='95% confidence')
ax.axhline(-condInt, color='r', linestyle='--')
ax.set_xlabel('Lag')
ax.set_ylabel('CCF')
ax.set_title('Cross-correlation between etilde and x')
plt.grid(True)
plt.show()

# x and etilde are not completely uncorrelated. Is this due to the input model issue or that we have the wrong model order? Unsure. 
# does transforming the data to x^{-1} improve this???

# Could it be an issue that temperature is not the primary driver behind energy consumption except in winter? 

# Still has very low fit to estimation data. Unsure how to improve this. 
amb_filt = signal.lfilter(model_1.B, model_1.F, ambient_diff_mdl)

fig, ax = plt.subplots()
ax.plot(amb_filt, label='amb')
ax.plot(power_diff_mdl, label='pow')
ax.legend()

# However, there still seems to be some explanatory power 


Discrete-time BJ model: y(t) = [B(z)/F(z)]x(t) + e(t)

B(z) = 2.0207(±0.2571)
F(z) = 1.0 - 0.9158(±0.0134)·z⁻¹

Polynomial orders: nB = 0    nF = 1
Number of free coefficients: 2
Fit to estimation data (NRMSE): 4.74%
FPE : 0.122  MSE : 0.122
AIC : 1135.687   BIC : 1146.37



In [164]:
# Plot etilde 
fig, ax = plt.subplots() 
ax.plot(etilde)

# Plot ACF and PACF to determine A1 and C1 orders 
plotACFnPACF(etilde)

# PACF has a clear spike at 1, ACF slowly declining. AR(1)? 
# ACF has a remaining spike at lag 24. We likely need an MA component for 24 here too 
# After this a small spike remains at lag 2. AR 2 or MA 2? 
# Neither works. 
p1 = 1
q1 = 26
cf1 = np.array([1] + [0]*23 + [1])
cf2 = np.array([1, 0, 1])
cf = np.convolve(cf1, cf2)
model2 = estimateARMA(etilde, A=p1, C=q1, C_free=cf, plot=False)
et = signal.lfilter(model2.A, model2.C, etilde[p1:])

plotACFnPACF(et, noLags=40, signLvl=0.05)
whiteness_test(et)
fig = plt.figure()
normplot(et)


Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 44.10 < 37.65)
  McLeod-Li test:        False (white if 113.16 < 37.65)
  Monti test:            False (white if 46.34 < 37.65)
  Sign change test:      False (white if 0.47 in [0.48,0.52])


In [165]:
# Calculate the full Box-Jenkins model given the above work. 
# We proceed with this hypothesis 
B_init = np.array([1])
A2_init = np.array([1, 1])
A1_init = np.array([1, 1])
C1_init = cf

# Same part are also free 
B_free = np.array([1])
A2_free = np.array([1, 1])
A1_free = np.array([1, 1])
C1_free = cf

# Calculate model 
pemslv = PEM(power_diff_mdl, ambient_diff_mdl, B=B_init, F=A2_init, C=C1_init, D=A1_init)
pemslv.set_free_params(B_free=B_free, F_free=A2_free, C_free=C1_free, D_free=A1_free)
model = pemslv.fit()
res = model.resid
model.summary()

plotACFnPACF(res)
whiteness_test(res)
check_if_normal(res)
plt.figure() 
normplot(res)

# Save the polynomials as the names from the book for easier use 
A1 = model.D 
A2 = model.F
C1 = model.C
B = model.B

# The results are close-ish. Could be down to missing input 

C:\Users\Johan\anaconda3\envs\timeseries\Lib\site-packages\scipy\optimize\_lsq\trf.py:527: RuntimeWarning: overflow encountered in dot
  cost_new = 0.5 * np.dot(f_new, f_new)


Discrete-time BJ model: y(t) = [B(z)/F(z)]x(t) + [C(z)/D(z)]e(t)

B(z) = 1.5232(±0.4159)
C(z) = 1.0 - 0.127(±0.0273)·z⁻² - 0.9018(±0.0112)·z⁻²⁴ + 0.1213(±0.0269)·z⁻²⁶
D(z) = 1.0 - 0.9316(±0.01)·z⁻¹
F(z) = 1.0 - 0.9592(±0.0124)·z⁻¹

Polynomial orders: nB = 0    nC = 26    nD = 1    nF = 1
Number of free coefficients: 6
Fit to estimation data (NRMSE): 65.22%
FPE : 0.016  MSE : 0.016
AIC : -1964.209   BIC : -1932.164

Whiteness test with 5.0% significance
  Ljung-Box-Pierce test: False (white if 41.91 < 37.65)
  McLeod-Li test:        False (white if 111.43 < 37.65)
  Monti test:            False (white if 43.32 < 37.65)
  Sign change test:      True (white if 0.48 in [0.48,0.52])
The D'Agostino-Pearson K2 test indicates that the data is NOT normal distributed.


In [171]:
# Attempt to make some predictions 
# Add the differentiation if possible, such that we do not have to do this separately 

k = 1 # Also k = 7 in final version 

# Form KA, KB and KC 
KA = np.convolve(A1, A2)
KB = np.convolve(A1, B)
KC = np.convolve(A2, C1)

# Predict 
Fx, Gx = polydiv(input_model.C, input_model.A, k) 

Fy, Gy = polydiv(C1, A1, k)
Fhh, Ghh = polydiv(np.convolve(Fy, KB), KC, k)

# Predict: 
xhatk_mdl = signal.lfilter(Gx, input_model.C, ambient_diff_mdl)
yhatk_mdl = signal.lfilter(Fhh, [1], xhatk_mdl) + signal.lfilter(Ghh, KC, ambient_diff_mdl) + signal.lfilter(Gy, KC, power_diff_mdl)

idx = np.max([Fhh.size, Ghh.size, Gy.size])
print(idx)
yhatk_mdl = yhatk_mdl[idx:]
err = power_diff_mdl[idx:] - yhatk_mdl

fig, ax = plt.subplots()
ax.plot(power_diff_mdl[idx:], label='true')
ax.plot(yhatk_mdl, label='pred')
ax.legend() 

# There is some mistake that needs to be fixed in the prediction 

28


[ 1.         -0.95918344 -0.12698896  0.12180571  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.90179674  0.8649885   0.12129749 -0.11634654]
